# Import Libraries

In [51]:
params = {}

In [52]:
import time
import json
import sys

from data_task_helpers import something

something()

from src.dte.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task, find_json_arg  # noqa: E402



something
['/Users/davidsongikandi/anaconda3/lib/python311.zip', '/Users/davidsongikandi/anaconda3/lib/python3.11', '/Users/davidsongikandi/anaconda3/lib/python3.11/lib-dynload', '', '/Users/davidsongikandi/duft-server/.venv/lib/python3.11/site-packages', '/Users/davidsongikandi/duft-server/duft']


In [53]:
   
environment = initialise_data_task("FM Sentinel Events Data Task", params=params)
params["name"] = params.get("name", "No parameters given!")
params["sleep_time"] = params.get("sleep_time", 0.2)

if not params:
    environment.log_error("No parameters given!")
    

[2024-03-03T12:20:10.423944] FM Sentinel Events Data Task - INFO: FM Sentinel Events Data Task initialised running in active mode
FM Sentinel Events Data Task: FM Sentinel Events Data Task initialised running in active mode


In [54]:
import jaydebeapi
import pandas as pd
from pandas import DataFrame
from sqlalchemy import create_engine
from jaydebeapi import Connection
import time
import datetime

# Database Connections 

In [55]:
#######################################
# vars: source-confs
#######################################
jdbc_url = "jdbc:filemaker://localhost/QuantumOuNickHealthCentre"
username = "Administrator"
password = "Qepms!@dmin"
driver_jar_path = "/Library/Java/Extensions/fmjdbc.jar"
driver_class = "com.filemaker.jdbc.Driver"

#######################################
# vars: target-database
#######################################
destination_db_host = 'localhost'
destination_db_port = 5433
destination_db_username = 'postgres'
destination_db_password = 'root'
destination_db_name = 'duft'

# Process data

In [56]:

def open_file_maker_connection():
    try:
        return jaydebeapi.connect(driver_class, jdbc_url, [username, password], driver_jar_path)
    except Exception as ex:
        return None


def close_file_maker_connection(con: Connection):
    if con is not None:
        con.close()


def fetch_file_maker_data(query: str, con: Connection):
    cursor = con.cursor()
    cursor.execute(query)
    data_rows = cursor.fetchall()
    cursor.close()
    return data_rows


class PostgresContext:
    def __init__(self, host, port, db, username, password):
        self.host = host
        self.db_port = port
        self.db_name = db
        self.db_username = username
        self.db_password = password
        self.engine = None

    def set_postgres_context(self):
        connection_url = ('postgresql://{0}:{1}@{2}:{3}/{4}'
                          .format(self.db_username, self.db_password, self.host, self.db_port, self.db_name))
        engine = create_engine(connection_url)
        self.engine = engine
        return self

    def get_dataframe_from_table(self, table_name, columns=None) -> DataFrame:
        if columns is None:
            columns = []
        df = pd.read_sql_table(table_name, con=self.engine, columns=columns)
        return df

    def write_dataframe_to_table(self, df: DataFrame, table_name: str):
        df.to_sql(table_name, con=self.engine, if_exists='replace', index=False, schema='analysis')
        return

# Define source and destination connections

In [57]:
fm_connection = open_file_maker_connection()

destination_context = PostgresContext(destination_db_host,
                                      destination_db_port,
                                      destination_db_name,
                                      destination_db_username,
                                      destination_db_password) \
    .set_postgres_context()

# Create dim_patient Data Frame

In [58]:
patient_confs = {
    "query": "SELECT id, sex, dob, maritalStatus FROM pat",
    "cols": ['patient_id', 'sex', 'dob', 'marital_status']
}

patient_query = patient_confs.get("query")
patient_cols = patient_confs.get("cols")

patient_data = fetch_file_maker_data(patient_query, fm_connection)
dim_pat_df = pd.DataFrame(patient_data, columns=patient_cols)

dim_pat_df['dob'] = pd.to_datetime(dim_pat_df['dob'])

dim_pat_df.head()

,patient_id,sex,dob,marital_status
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,Female,1990-01-02,Single
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,Female,1931-02-03,Single
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,Female,1964-12-04,Married
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,Female,1999-02-02,Single
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,Female,1996-04-09,Single


# Create fact_hiv_diagnosis Data Frame

In [59]:
fact_hiv_diagnosis_confs = {
    "query": "SELECT idpatient, hivconfirmationdate, hivconfirmatoryresultsdate, hivconfirmatoryresultstype, idfacilitycreate FROM Cd",
    "cols": ['patient_id', 'hiv_confirmation_date','hiv_confirmatory_result_date', 'hiv_confirmatory_result_type', 'hiv_diagnosis_facility_id']
}

fact_hiv_diagnosis_query = fact_hiv_diagnosis_confs.get("query")
fact_hiv_diagnosis_cols = fact_hiv_diagnosis_confs.get("cols")

fact_hiv_diagnosis_data = fetch_file_maker_data(fact_hiv_diagnosis_query, fm_connection)
fact_hiv_diagnosis_df = pd.DataFrame(fact_hiv_diagnosis_data, columns=fact_hiv_diagnosis_cols)

fact_hiv_diagnosis_df['hiv_confirmation_date'] = pd.to_datetime(fact_hiv_diagnosis_df['hiv_confirmation_date'])
fact_hiv_diagnosis_df['hiv_confirmatory_result_date'] = pd.to_datetime(fact_hiv_diagnosis_df['hiv_confirmatory_result_date'])

fact_hiv_diagnosis_df.head()

,patient_id,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,NaT,NaT,None,22EE64B5-6A63-4FFE-9413-AE421220D884
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,2004-03-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,2004-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,2004-07-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,2002-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6


# Add dim_patient, fact_hiv_diagnosis to fact_sentinel_event dataframe

In [60]:
fact_sentinel_event_df = pd.merge(dim_pat_df, fact_hiv_diagnosis_df, on='patient_id', how='left')
fact_sentinel_event_df.head()

,patient_id,sex,dob,marital_status,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,Female,1990-01-02,Single,NaT,NaT,None,22EE64B5-6A63-4FFE-9413-AE421220D884
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,Female,1931-02-03,Single,2004-03-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,Female,1964-12-04,Married,2004-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,Female,1999-02-02,Single,2004-07-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,Female,1996-04-09,Single,2002-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6


# Create hiv_enrolment dataframe 

In [61]:
fact_hiv_enrolment_confs = {
    "query": "SELECT c.idpatient, c.idfacilitycreate, c.hivenrolleddate, c.disclosureenrollmentdate, c.arteligiblereason, c.artstartdate, p.artnumber, p.artnumberlegacy FROM pat p INNER JOIN Cd c ON p.id=c.idpatient",
    "cols": ['patient_id', 'hiv_enrollment_facility_id', 'hiv_enrollment_date', 'hiv_disclosure_enrollment_date','art_eligibility_reason','art_start_date', 'art_number', 'art_number_legacy']
}

fact_hiv_enrolment_query = fact_hiv_enrolment_confs.get("query")
fact_hiv_enrolment_cols = fact_hiv_enrolment_confs.get("cols")

fact_hiv_enrolment_data = fetch_file_maker_data(fact_hiv_enrolment_query, fm_connection)
fact_hiv_enrolment_df = pd.DataFrame(fact_hiv_enrolment_data, columns=fact_hiv_enrolment_cols)

fact_hiv_enrolment_df['hiv_enrollment_date'] = pd.to_datetime(fact_hiv_enrolment_df['hiv_enrollment_date'])
fact_hiv_enrolment_df['hiv_disclosure_enrollment_date'] = pd.to_datetime(fact_hiv_enrolment_df['hiv_disclosure_enrollment_date'])
fact_hiv_enrolment_df['art_start_date'] = pd.to_datetime(fact_hiv_enrolment_df['art_start_date'])

fact_hiv_enrolment_df.head()

,patient_id,hiv_enrollment_facility_id,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligibility_reason,art_start_date,art_number,art_number_legacy
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,22EE64B5-6A63-4FFE-9413-AE421220D884,NaT,NaT,None,NaT,Unassigned,2180402000024
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-15,NaT,Clinical & CD4,2004-09-15,99999-0904-1,127090402008
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-30,NaT,Clinical & CD4,2004-09-16,99999-0904-2,127090402154
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-12-15,NaT,Clinical only,2004-10-09,99999-1204-3,127120402855
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-07-16,NaT,Clinical & CD4,2004-07-16,99999-0704-4,127070401247


# Add fact_hiv_enrolment to fact_sentinel_event dataframe

In [62]:
fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_hiv_enrolment_df, on='patient_id', how='left')
fact_sentinel_event_df.head()

,patient_id,sex,dob,marital_status,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,hiv_enrollment_facility_id,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligibility_reason,art_start_date,art_number,art_number_legacy
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,Female,1990-01-02,Single,NaT,NaT,None,22EE64B5-6A63-4FFE-9413-AE421220D884,22EE64B5-6A63-4FFE-9413-AE421220D884,NaT,NaT,None,NaT,Unassigned,2180402000024
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,Female,1931-02-03,Single,2004-03-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-15,NaT,Clinical & CD4,2004-09-15,99999-0904-1,127090402008
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,Female,1964-12-04,Married,2004-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-30,NaT,Clinical & CD4,2004-09-16,99999-0904-2,127090402154
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,Female,1999-02-02,Single,2004-07-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-12-15,NaT,Clinical only,2004-10-09,99999-1204-3,127120402855
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,Female,1996-04-09,Single,2002-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-07-16,NaT,Clinical & CD4,2004-07-16,99999-0704-4,127070401247


# Create fact_visits dataframe 

In [63]:
fact_visits_confs = {
    "query": "SELECT idpatient, idfacilitycreate, visitDate, followupDate, scheduledDate, careModel, pregnantstatus, Breastfeeding, pregnantlmp, pregnantedd, ctxadherence, arvadherence FROM Fup",
    "cols": ['patient_id', 'visit_facility_id', 'visit_date', 'next_visit_date', 'scheduled_visit_date', 'care_model', 'pregnancy_status', 'breast_feeding', 'lmp', 'edd', 'ctx_adherence', 'arv_adherence' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

fact_visits_data = fetch_file_maker_data(fact_visits_query, fm_connection)
fact_visits_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

fact_visits_df['visit_date'] = pd.to_datetime(fact_visits_df['visit_date'])
fact_visits_df['next_visit_date'] = pd.to_datetime(fact_visits_df['next_visit_date'], errors='coerce')

fact_visits_df.head()

,patient_id,visit_facility_id,visit_date,next_visit_date,scheduled_visit_date,care_model,pregnancy_status,breast_feeding,lmp,edd,ctx_adherence,arv_adherence
0,D9699CEF-4AE5-0C4D-9446-ABBE52BA219E,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2010-07-02,2010-10-22,None,None,None,None,None,None,G,G
1,83590047-7856-7A48-9877-8A250C5A8EA8,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2004-11-18,2005-02-10,None,None,None,None,None,None,None,G
2,D10F8829-5103-DB49-927B-DA3759A7C3E1,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2008-06-03,2008-08-27,None,None,None,None,None,None,G,G
3,DB22C3B0-13E7-514C-B5FD-B6BF959BD619,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2007-10-09,2007-12-04,None,None,None,None,None,None,G,G
4,5C1ED366-618F-A142-B069-D1C4C49B69D2,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2008-12-15,2009-01-13,None,None,None,None,None,None,G,G


# Create fact_first_visit dataframe 

In [64]:
fact_first_visit_df = fact_visits_df.loc[fact_visits_df.groupby('patient_id')['visit_date'].idxmin()].reset_index(drop=True)
fact_first_visit_df.head()

,patient_id,visit_facility_id,visit_date,next_visit_date,scheduled_visit_date,care_model,pregnancy_status,breast_feeding,lmp,edd,ctx_adherence,arv_adherence
0,0006DC70-852D-3647-9CCA-D84FBEDCA753,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2017-01-10,2017-04-04,None,None,None,None,None,None,G,G
1,000EC4B2-0174-F34D-89AC-CBC0F206E504,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2019-04-23,2019-06-26,None,None,None,None,None,None,G,G
2,002AB39F-652A-B74D-868B-065E29DD48A5,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2021-09-22,2021-10-21,2021-10-06,None,None,None,None,None,Good,Good
3,004E6E18-69E0-4A4A-AFEE-412B062C77D8,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2017-02-02,2017-02-17,None,None,None,None,None,None,G,G
4,006C71E9-6941-984C-9DED-34E0EE8357EC,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2017-05-11,2017-05-26,None,None,None,None,None,None,G,G


# Add first visit info to fact_sentinel_event

In [65]:
columns_to_add = {
    'visit_date': 'first_visit_date', 
    'visit_facility_id': 'first_visit_facility_id'
}

fact_first_visits_renamed_df = fact_first_visit_df[list(columns_to_add.keys()) + ['patient_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_visits_renamed_df, on='patient_id', how='left')
fact_sentinel_event_df.head()

,patient_id,sex,dob,marital_status,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,hiv_enrollment_facility_id,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligibility_reason,art_start_date,art_number,art_number_legacy,first_visit_date,first_visit_facility_id
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,Female,1990-01-02,Single,NaT,NaT,None,22EE64B5-6A63-4FFE-9413-AE421220D884,22EE64B5-6A63-4FFE-9413-AE421220D884,NaT,NaT,None,NaT,Unassigned,2180402000024,NaT,NaN
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,Female,1931-02-03,Single,2004-03-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-15,NaT,Clinical & CD4,2004-09-15,99999-0904-1,127090402008,2012-10-16,81E613CF-2271-4A95-8921-E1DDB3BAE9A4
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,Female,1964-12-04,Married,2004-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-30,NaT,Clinical & CD4,2004-09-16,99999-0904-2,127090402154,2012-11-13,81E613CF-2271-4A95-8921-E1DDB3BAE9A4
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,Female,1999-02-02,Single,2004-07-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-12-15,NaT,Clinical only,2004-10-09,99999-1204-3,127120402855,2012-09-21,81E613CF-2271-4A95-8921-E1DDB3BAE9A4
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,Female,1996-04-09,Single,2002-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-07-16,NaT,Clinical & CD4,2004-07-16,99999-0704-4,127070401247,2012-05-15,81E613CF-2271-4A95-8921-E1DDB3BAE9A4


# Create fact_last_visit dataframe 

In [66]:
fact_last_visit_df = fact_visits_df.loc[fact_visits_df.groupby('patient_id')['visit_date'].idxmax()].reset_index(drop=True)
fact_last_visit_df.head()

,patient_id,visit_facility_id,visit_date,next_visit_date,scheduled_visit_date,care_model,pregnancy_status,breast_feeding,lmp,edd,ctx_adherence,arv_adherence
0,0006DC70-852D-3647-9CCA-D84FBEDCA753,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2018-07-13,2018-08-10,2017-04-04,None,None,None,None,None,G,G
1,000EC4B2-0174-F34D-89AC-CBC0F206E504,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2019-07-16,2019-07-31,2019-06-26,None,None,None,None,None,G,G
2,002AB39F-652A-B74D-868B-065E29DD48A5,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2022-03-03,2022-04-01,2022-03-03,None,No,None,None,None,Good,Good
3,004E6E18-69E0-4A4A-AFEE-412B062C77D8,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2022-02-08,2022-06-28,2022-02-08,None,None,None,None,None,Good,Good
4,006C71E9-6941-984C-9DED-34E0EE8357EC,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2021-10-04,2022-01-24,2021-11-03,None,None,None,None,None,Good,Good


# Add last visit info to fact_sentinel_event

In [67]:
columns_to_add = {
    'visit_date': 'last_visit_date',
    'next_visit_date': 'last_next_visit_date', 
    'scheduled_visit_date': 'last_scheduled_visit_date', 
    'pregnancy_status': 'last_pregnancy_status', 
    'breast_feeding': 'last_breast_feeding', 
    'lmp': 'last_lmp', 
    'edd': 'last_edd',
    'visit_facility_id': 'last_visit_facility_id'
}

fact_last_viral_load_renamed_df = fact_last_visit_df[list(columns_to_add.keys()) + ['patient_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_viral_load_renamed_df, on='patient_id', how='left')

fact_sentinel_event_df['last_visit_duration'] = (fact_sentinel_event_df['last_next_visit_date'] - fact_sentinel_event_df['last_visit_date']).dt.days

fact_sentinel_event_df.head()

,patient_id,sex,dob,marital_status,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,hiv_enrollment_facility_id,hiv_enrollment_date,...,first_visit_facility_id,last_visit_date,last_next_visit_date,last_scheduled_visit_date,last_pregnancy_status,last_breast_feeding,last_lmp,last_edd,last_visit_facility_id,last_visit_duration
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,Female,1990-01-02,Single,NaT,NaT,None,22EE64B5-6A63-4FFE-9413-AE421220D884,22EE64B5-6A63-4FFE-9413-AE421220D884,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,Female,1931-02-03,Single,2004-03-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-15,...,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2022-04-20,2022-10-05,2022-04-20,Yes,Yes,2022-06-21,2023-03-18,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,168.0
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,Female,1964-12-04,Married,2004-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-30,...,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2022-02-22,2022-06-14,2022-02-22,No,None,None,None,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,112.0
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,Female,1999-02-02,Single,2004-07-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-12-15,...,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2020-08-04,2020-10-26,2020-08-04,No,None,None,None,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,83.0
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,Female,1996-04-09,Single,2002-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-07-16,...,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2022-04-21,2022-06-16,2022-04-23,None,None,None,None,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,56.0


# Create fact_tpt dataframe 

In [68]:
fact_tpt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, regimen, startDate, duration, stopDateExpected, stopDateActual , outcome FROM tbt",
    "cols": ['patient_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration', 'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_outcome']
}

fact_tpt_query = fact_tpt_confs.get("query")
fact_tpt_cols = fact_tpt_confs.get("cols")

fact_tpt_data = fetch_file_maker_data(fact_tpt_query, fm_connection)
fact_tpt_df = pd.DataFrame(fact_tpt_data, columns=fact_tpt_cols)

fact_tpt_df['tpt_start_date'] = pd.to_datetime(fact_tpt_df['tpt_start_date'])
fact_tpt_df['tpt_expected_stop_date'] = pd.to_datetime(fact_tpt_df['tpt_expected_stop_date'])
fact_tpt_df['tpt_actual_stop_date'] = pd.to_datetime(fact_tpt_df['tpt_actual_stop_date'])

fact_tpt_df.head()

,patient_id,tpt_facility_id,tpt_regimen,tpt_start_date,tpt_duration,tpt_expected_stop_date,tpt_actual_stop_date,tpt_outcome
0,83590047-7856-7A48-9877-8A250C5A8EA8,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,None,2017-04-28,6.0,2017-10-28,NaT,None
1,52A77F13-4A98-9C41-BA4E-FDED4AA03818,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,None,2015-07-14,6.0,2016-01-14,NaT,None
2,7F09AB96-1CF8-BF4D-AB2A-F15D18207EE1,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,None,2006-04-20,6.0,2006-10-20,2010-02-18,None
3,933244A3-1443-C449-9978-F16B5F2E4861,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,None,2006-06-06,6.0,2006-12-06,2007-01-03,None
4,DB1478F1-216D-1B4E-8FD7-ECA52CEC0AE1,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,None,2015-12-14,6.0,2016-06-14,NaT,None


# Add tpt info to fact_sentinel_event

In [69]:
columns_to_add = ['patient_id','tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration', 'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_outcome']

fact_tpt_df_selected_df = fact_tpt_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_tpt_df_selected_df, on='patient_id', how='left')
fact_sentinel_event_df.head()

,patient_id,sex,dob,marital_status,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,hiv_enrollment_facility_id,hiv_enrollment_date,...,last_edd,last_visit_facility_id,last_visit_duration,tpt_facility_id,tpt_regimen,tpt_start_date,tpt_duration,tpt_expected_stop_date,tpt_actual_stop_date,tpt_outcome
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,Female,1990-01-02,Single,NaT,NaT,None,22EE64B5-6A63-4FFE-9413-AE421220D884,22EE64B5-6A63-4FFE-9413-AE421220D884,NaT,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,Female,1931-02-03,Single,2004-03-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-15,...,2023-03-18,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,168.0,NaN,NaN,NaT,NaN,NaT,NaT,NaN
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,Female,1964-12-04,Married,2004-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-30,...,None,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,112.0,NaN,NaN,NaT,NaN,NaT,NaT,NaN
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,Female,1999-02-02,Single,2004-07-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-12-15,...,None,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,83.0,NaN,NaN,NaT,NaN,NaT,NaT,NaN
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,Female,1996-04-09,Single,2002-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-07-16,...,None,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,56.0,NaN,NaN,NaT,NaN,NaT,NaT,NaN


# Create fact_lab dataframe 

In [70]:
fact_lab_confs = {
    "query": "SELECT l.idpatient, l.idfacilitycreate, l.orderDate, l.result, l.resultDate, lm.name FROM lab l INNER JOIN LabMaster lm ON lm.id=l.idtest",
    "cols": ['patient_id', 'lab_facility_id', 'order_date', 'lab_result', 'result_date', 'test_name']
}

fact_lab_query = fact_lab_confs.get("query")
fact_lab_cols = fact_lab_confs.get("cols")

fact_lab_data = fetch_file_maker_data(fact_lab_query, fm_connection)
fact_lab_df = pd.DataFrame(fact_lab_data, columns=fact_lab_cols)

fact_lab_df.head()

,patient_id,lab_facility_id,order_date,lab_result,result_date,test_name
0,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2004-09-15,65.0,2004-09-15,CD4
1,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2004-09-30,49.0,2004-09-30,CD4
2,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2004-12-15,16.8,2004-12-15,CD4
3,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2004-07-16,6.4,2004-07-16,CD4
4,A27A9A42-3B6B-4B49-BFB6-6B387EC68659,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2007-03-09,99.0,2007-03-09,CD4


# Create fact_first_viral_load dataframe

In [71]:
vl_tests_df = fact_lab_df[fact_lab_df['test_name'] == 'Viral Load']
fact_first_viral_load_tests_df = vl_tests_df.sort_values('order_date').groupby('patient_id').first().reset_index()

fact_first_viral_load_tests_df['lab_result'] = pd.to_numeric(fact_first_viral_load_tests_df['lab_result'], errors='coerce')

def categorize_result(result):
    if result < 40:
        return 'Suppressed'
    elif 40 <= result <= 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

# Add the category column based on the conditions
fact_first_viral_load_tests_df['category'] = fact_first_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_first_viral_load_tests_df.head()

,patient_id,lab_facility_id,order_date,lab_result,result_date,test_name,category
0,000EC4B2-0174-F34D-89AC-CBC0F206E504,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2019-05-22,0.000,2019-05-22,Viral Load,Suppressed
1,004E6E18-69E0-4A4A-AFEE-412B062C77D8,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2018-06-06,0.000,2018-06-06,Viral Load,Suppressed
2,006C71E9-6941-984C-9DED-34E0EE8357EC,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2018-08-17,119757.611,2018-08-17,Viral Load,Unsuppressed
3,00963507-6A4D-EA45-BD42-DF3E8E3F8B06,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2018-06-12,0.000,2018-06-12,Viral Load,Suppressed
4,00A6846B-7234-CE47-B334-9D0FA41F3D05,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2017-04-11,0.000,2017-04-11,Viral Load,Suppressed


# Create fact_last_viral_load dataframe

In [72]:
fact_last_viral_load_tests_df = vl_tests_df.sort_values('order_date').groupby('patient_id').last().reset_index()
fact_last_viral_load_tests_df['lab_result'] = pd.to_numeric(fact_last_viral_load_tests_df['lab_result'], errors='coerce')

def categorize_result(result):
    if result < 40:
        return 'Suppressed'
    elif 40 <= result <= 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

# Add the category column based on the conditions
fact_last_viral_load_tests_df['category'] = fact_last_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_last_viral_load_tests_df.head()

,patient_id,lab_facility_id,order_date,lab_result,result_date,test_name,category
0,000EC4B2-0174-F34D-89AC-CBC0F206E504,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2019-05-22,0.0,2019-05-22,Viral Load,Suppressed
1,004E6E18-69E0-4A4A-AFEE-412B062C77D8,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2021-06-29,0.0,2021-06-29,Viral Load,Suppressed
2,006C71E9-6941-984C-9DED-34E0EE8357EC,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2019-06-13,4551.0,2019-06-13,Viral Load,Unsuppressed
3,00963507-6A4D-EA45-BD42-DF3E8E3F8B06,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2021-06-30,0.0,2019-06-12,Viral Load,Suppressed
4,00A6846B-7234-CE47-B334-9D0FA41F3D05,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,2021-02-10,0.0,2021-02-10,Viral Load,Suppressed


# Add first viral load info to fact_sentinel_event

In [73]:
columns_to_add = {
    'order_date': 'first_viral_load_order_date',
    'result_date': 'first_viral_load_result_date', 
    'lab_result': 'first_viral_load_result',
    'lab_facility_id': 'first_viral_load_facility_id',
    'category': 'first_viral_load_result_category'
}

fact_first_viral_load_renamed_df = fact_first_viral_load_tests_df[list(columns_to_add.keys()) + ['patient_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_viral_load_renamed_df, on='patient_id', how='left')

fact_sentinel_event_df.head()

,patient_id,sex,dob,marital_status,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,hiv_enrollment_facility_id,hiv_enrollment_date,...,tpt_start_date,tpt_duration,tpt_expected_stop_date,tpt_actual_stop_date,tpt_outcome,first_viral_load_order_date,first_viral_load_result_date,first_viral_load_result,first_viral_load_facility_id,first_viral_load_result_category
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,Female,1990-01-02,Single,NaT,NaT,None,22EE64B5-6A63-4FFE-9413-AE421220D884,22EE64B5-6A63-4FFE-9413-AE421220D884,NaT,...,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,Female,1931-02-03,Single,2004-03-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-15,...,NaT,NaN,NaT,NaT,NaN,2013-05-28,2013-05-28,183.39,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,LLV
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,Female,1964-12-04,Married,2004-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-30,...,NaT,NaN,NaT,NaT,NaN,2017-04-10,2017-04-10,0.00,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,Female,1999-02-02,Single,2004-07-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-12-15,...,NaT,NaN,NaT,NaT,NaN,2017-08-02,2017-08-02,0.00,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,Female,1996-04-09,Single,2002-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-07-16,...,NaT,NaN,NaT,NaT,NaN,2015-02-17,2015-02-17,0.00,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed


# Add last viral load info to fact_sentinel_event

In [74]:
columns_to_add = {
    'order_date': 'last_viral_load_order_date',
    'result_date': 'last_viral_load_result_date', 
    'lab_result': 'last_viral_load_result',
    'lab_facility_id': 'last_viral_load_facility_id',
    'category': 'last_viral_load_result_category'
}

fact_last_viral_load_renamed_df = fact_last_viral_load_tests_df[list(columns_to_add.keys()) + ['patient_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_viral_load_renamed_df, on='patient_id', how='left')

fact_sentinel_event_df.head()

,patient_id,sex,dob,marital_status,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,hiv_enrollment_facility_id,hiv_enrollment_date,...,first_viral_load_order_date,first_viral_load_result_date,first_viral_load_result,first_viral_load_facility_id,first_viral_load_result_category,last_viral_load_order_date,last_viral_load_result_date,last_viral_load_result,last_viral_load_facility_id,last_viral_load_result_category
0,C0199AF2-4115-CC46-B3FF-1C41AD2DBEBE,Female,1990-01-02,Single,NaT,NaT,None,22EE64B5-6A63-4FFE-9413-AE421220D884,22EE64B5-6A63-4FFE-9413-AE421220D884,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D5A04EAE-3782-8C47-97BB-BE6BB6506B90,Female,1931-02-03,Single,2004-03-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-15,...,2013-05-28,2013-05-28,183.39,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,LLV,2022-01-26,2022-01-26,0.0,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed
2,EFEB6CAB-4A6E-754D-A7BA-1368B831722D,Female,1964-12-04,Married,2004-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-09-30,...,2017-04-10,2017-04-10,0.00,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed,2022-06-22,2022-06-22,0.0,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed
3,E5DACB9D-AAAB-C341-8E41-B6D73C642A5B,Female,1999-02-02,Single,2004-07-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-12-15,...,2017-08-02,2017-08-02,0.00,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed,2019-10-07,2019-10-07,0.0,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed
4,2018E8D9-823D-6540-B6CB-A4AB4A3BF3BF,Female,1996-04-09,Single,2002-09-01,NaT,None,1B258671-AF7E-4872-A207-1C10E742A8F6,1B258671-AF7E-4872-A207-1C10E742A8F6,2004-07-16,...,2015-02-17,2015-02-17,0.00,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed,2021-08-12,2021-08-12,0.0,81E613CF-2271-4A95-8921-E1DDB3BAE9A4,Suppressed


# Write sentinel event to table

In [75]:
destination_context.write_dataframe_to_table(fact_sentinel_event_df, 'fact_sentinel_event')